<a href="https://colab.research.google.com/github/mashaleo/NEEC/blob/main/Customer_Complaints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv('balanced_complaints.csv')
df_no_text = df.drop(columns=['Consumer complaint narrative'])
print(df_no_text.head())

  Date received          Product                                  Issue  \
0    2016-03-25  Debt collection     False statements or representation   
1    2015-05-08  Debt collection  Cont'd attempts collect debt not owed   
2    2016-01-21  Debt collection                  Communication tactics   
3    2015-05-01  Debt collection  Cont'd attempts collect debt not owed   
4    2015-09-26      Credit card                   Credit determination   

                             Company public response Date sent to company  \
0  Company believes it acted appropriately as aut...           2016-03-26   
1  Company can't verify or dispute the facts in t...           2015-05-11   
2  Company can't verify or dispute the facts in t...           2016-01-21   
3  Company believes it acted appropriately as aut...           2015-05-01   
4   Company chooses not to provide a public response           2015-09-29   

      Company response to consumer Timely response?  
0          Closed with explanati

In [10]:
product_counts = df['Product'].value_counts().nunique()
product_unique = df['Product'].unique()
print(product_unique)
print(product_counts)

issue_counts = df['Issue'].value_counts().nunique()
issue_unique = df['Issue'].unique()
print(issue_unique)
print(issue_counts)

public_response_counts = df['Company public response'].value_counts().nunique()
public_response_unique = df['Company public response'].unique()
print(public_response_unique)
print(public_response_counts)

consumer_response_counts = df['Company response to consumer'].value_counts().nunique()
consumer_response_unique = df['Company response to consumer'].unique()
print(consumer_response_unique)
print(consumer_response_counts)

['Debt collection' 'Credit card'
 'Payday loan, title loan, or personal loan' 'Mortgage'
 'Credit card or prepaid card' 'Credit reporting'
 'Checking or savings account' 'Vehicle loan or lease'
 'Credit reporting, credit repair services, or other personal consumer reports'
 'Consumer Loan' 'Money transfer, virtual currency, or money service'
 'Bank account or service' 'Student loan' 'Money transfers' 'Payday loan'
 'Credit reporting or other personal consumer reports'
 'Other financial service' 'Prepaid card'
 'Payday loan, title loan, personal loan, or advance loan'
 'Debt or credit management']
19
['False statements or representation'
 "Cont'd attempts collect debt not owed" 'Communication tactics'
 'Credit determination' "Charged fees or interest you didn't expect"
 'Disclosure verification of debt'
 'Loan servicing, payments, escrow account' 'Fees or interest'
 'Settlement process and costs' 'Taking/threatening an illegal action'
 'Written notification about debt' 'Trouble during p

In [4]:
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=df['Date received'].min(), end=df['Date sent to company'].max())

def calculate_business_days(row):
    # Generate business days between the two dates
    business_days = pd.bdate_range(row['Date received'], row['Date sent to company'])
    # Exclude holidays
    valid_business_days = business_days.difference(holidays)
    return len(valid_business_days)
df['Business Days'] = df.apply(calculate_business_days, axis=1)
print(df.head())

  Date received          Product                                  Issue  \
0    2016-03-25  Debt collection     False statements or representation   
1    2015-05-08  Debt collection  Cont'd attempts collect debt not owed   
2    2016-01-21  Debt collection                  Communication tactics   
3    2015-05-01  Debt collection  Cont'd attempts collect debt not owed   
4    2015-09-26      Credit card                   Credit determination   

                        Consumer complaint narrative  \
0  Debt amounts are not accurate when attempting ...   
1  Focus Receivables Management, XXXX, GA XXXX Ph...   
2  My mother, my sister, and I have received mult...   
3  I started to receive correspondence from Natio...   
4                      XXXX ruined my credit rating.   

                             Company public response Date sent to company  \
0  Company believes it acted appropriately as aut...           2016-03-26   
1  Company can't verify or dispute the facts in t...      

In [13]:
mean_business_days = df['Business Days'].mean()
median_business_days = df['Business Days'].median()
max_business_days = df['Business Days'].max()
min_business_days = df['Business Days'].min()
standard_dev = df['Business Days'].std()

print(mean_business_days)
print(median_business_days)
print(max_business_days)
print(min_business_days)
print(standard_dev)

2.7161387220098305
1.0
172
0
7.602446586540142


In [18]:
average_business_days_per_product = df.groupby('Product')['Business Days'].mean()
print(average_business_days_per_product)

Product
Bank account or service                                                         1.547231
Checking or savings account                                                     1.445298
Consumer Loan                                                                   3.668966
Credit card                                                                     1.473913
Credit card or prepaid card                                                     1.509569
Credit reporting                                                                2.392694
Credit reporting or other personal consumer reports                             0.981481
Credit reporting, credit repair services, or other personal consumer reports    1.902903
Debt collection                                                                 3.359347
Debt or credit management                                                       1.000000
Money transfer, virtual currency, or money service                              3.235772
Money transfe

In [19]:
average_business_days_per_issue = df.groupby('Issue')['Business Days'].mean()
print(average_business_days_per_issue)

Issue
APR or interest rate                                        1.153846
Account opening, closing, or management                     1.537190
Advertising                                                 1.000000
Advertising and marketing                                   3.000000
Advertising and marketing, including promotional offers     0.857143
                                                             ...    
Using a debit or ATM card                                   0.928571
Vehicle was repossessed or sold the vehicle                 0.000000
Was approved for a loan, but didn't receive the money      21.000000
Written notification about debt                             2.383513
Wrong amount charged or received                            6.000000
Name: Business Days, Length: 140, dtype: float64


In [20]:
average_business_days_per_response = df.groupby('Company public response')['Business Days'].mean()
print(average_business_days_per_issue)

Issue
APR or interest rate                                        1.153846
Account opening, closing, or management                     1.537190
Advertising                                                 1.000000
Advertising and marketing                                   3.000000
Advertising and marketing, including promotional offers     0.857143
                                                             ...    
Using a debit or ATM card                                   0.928571
Vehicle was repossessed or sold the vehicle                 0.000000
Was approved for a loan, but didn't receive the money      21.000000
Written notification about debt                             2.383513
Wrong amount charged or received                            6.000000
Name: Business Days, Length: 140, dtype: float64


In [21]:
average_company_response = df.groupby('Company response to consumer')['Business Days'].mean()
print(average_company_response)

Company response to consumer
Closed                             3.890411
Closed with explanation            3.017031
Closed with monetary relief        1.969919
Closed with non-monetary relief    2.210422
Name: Business Days, dtype: float64


In [22]:
# Group by product and issue, then get unique issues for each product
unique_issues_per_product = df.groupby('Product')['Issue'].unique()
print(unique_issues_per_product)

Product
Bank account or service                                                         [Account opening, closing, or management, Depo...
Checking or savings account                                                     [Opening an account, Managing an account, Cred...
Consumer Loan                                                                   [Taking out the loan or lease, Managing the lo...
Credit card                                                                     [Credit determination, Billing disputes, APR o...
Credit card or prepaid card                                                     [Fees or interest, Problem with a purchase sho...
Credit reporting                                                                [Improper use of my credit report, Incorrect i...
Credit reporting or other personal consumer reports                             [Incorrect information on your report, Problem...
Credit reporting, credit repair services, or other personal consumer reports    [I